## Augmentation

In [1]:
import os
import random
from PIL import Image, ImageEnhance, ImageOps
import numpy as np

In [2]:
# Define augmentation functions
def random_rotation(image):
    angle = random.uniform(-30, 30)  # Rotate between -30 to 30 degrees
    return image.rotate(angle)

def random_flip(image):
    if random.choice([True, False]):
        return ImageOps.mirror(image)
    return image

def random_brightness(image):
    enhancer = ImageEnhance.Brightness(image)
    factor = random.uniform(0.7, 1.3)  # Brightness factor
    return enhancer.enhance(factor)

def random_contrast(image):
    enhancer = ImageEnhance.Contrast(image)
    factor = random.uniform(0.7, 1.3)  # Contrast factor
    return enhancer.enhance(factor)

def add_random_noise(image):
    np_image = np.array(image)
    noise = np.random.normal(0, 25, np_image.shape).astype(np.int16)
    noisy_image = np.clip(np_image + noise, 0, 255).astype(np.uint8)
    return Image.fromarray(noisy_image)

def augment_image(image):
    image = random_rotation(image)
    image = random_flip(image)
    image = random_brightness(image)
    image = random_contrast(image)
    image = add_random_noise(image)
    return image


In [3]:
input_folder = "C:/Users/User/OneDrive/Documents/SignLanguageApp/SLangDataset/mergedData2"
output_folder = "C:/Users/User/OneDrive/Documents/SignLanguageApp/SLangDataset/augmentedData2"
os.makedirs(output_folder, exist_ok=True)

In [5]:
# Iterate over all subfolders and images
for subdir, _, files in os.walk(input_folder):
    relative_path = os.path.relpath(subdir, input_folder)
    output_subdir = os.path.join(output_folder, relative_path)
    os.makedirs(output_subdir, exist_ok=True)

    for file in files:
        if file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'tiff')):
            input_path = os.path.join(subdir, file)
            output_path = os.path.join(output_subdir, file)

            try:
                with Image.open(input_path) as img:
                    img = img.convert("L")  # Ensure greyscale (black and white)
                    augmented_img = augment_image(img)
                    augmented_img.save(output_path)
            except Exception as e:
                print(f"Error processing {input_path}: {e}")

print("Data augmentation completed!")


Data augmentation completed!


## Training

### MobileNetV2

In [29]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import os

# Define the dataset path
data_dir = r"C:\Users\User\OneDrive\Documents\SignLanguageApp\SLangDataset\augmentedData"

# Parameters
img_size = (224, 224)  # Image size for resizing
batch_size = 32
num_classes = len(os.listdir(data_dir))  # Number of classes inferred from folders

# Train-test-validation split
train_datagen = ImageDataGenerator(
    rescale=1./255, validation_split=0.2  # Rescale pixel values and split
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Model selection: MobileNetV2 (simpler and lightweight)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Freeze the base model
base_model.trainable = False

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
epochs = 10
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=validation_generator.samples // batch_size
)

# Unfreeze and fine-tune the base model (optional)
base_model.trainable = True
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
fine_tune_epochs = 5
history_fine_tune = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=fine_tune_epochs,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=validation_generator.samples // batch_size
)

# Save the model
model.save("asl_sign_language_model_mobilenet.h5")

# Evaluate on validation set
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")


Found 43120 images belonging to 27 classes.
Found 10780 images belonging to 27 classes.
Epoch 1/10
1347/1347 [==============================] - 240s 176ms/step - batch: 673.0000 - size: 31.9881 - loss: 0.3599 - accuracy: 0.8953 - val_loss: 3.5143 - val_accuracy: 0.3595
Epoch 2/10
1347/1347 [==============================] - 241s 179ms/step - batch: 673.0000 - size: 31.9881 - loss: 0.1329 - accuracy: 0.9549 - val_loss: 4.7407 - val_accuracy: 0.3295
Epoch 3/10
1347/1347 [==============================] - 249s 185ms/step - batch: 673.0000 - size: 31.9881 - loss: 0.1077 - accuracy: 0.9635 - val_loss: 5.3248 - val_accuracy: 0.3370
Epoch 4/10
1347/1347 [==============================] - 245s 182ms/step - batch: 673.0000 - size: 31.9881 - loss: 0.0906 - accuracy: 0.9681 - val_loss: 5.8540 - val_accuracy: 0.3376
Epoch 5/10
1347/1347 [==============================] - 249s 185ms/step - batch: 673.0000 - size: 31.9881 - loss: 0.0878 - accuracy: 0.9688 - val_loss: 5.1617 - val_accuracy: 0.3797
Ep